# Trading Chatbot Demo Notebook
This notebook imports every `src` module to keep the logic in reusable `.py` files while showcasing the end-to-end flow for professors.

In [ ]:
from pathlib import Path
import sys

import pandas as pd
import plotly.graph_objects as go
from IPython.display import display

ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.utils import read_tickers
from src.data.fetchers import fetch_ohlcv
from src.ta_engine import compute_indicators, detect_breakout, detect_double_top_bottom
from src.fa_engine import evaluate_fundamentals
from src.strategy_engine import rank_strategies
from src.chat_agent import ChatAgent

pd.set_option("display.float_format", "{:,.2f}".format)
tickers = read_tickers(limit=3)
tickers

## 1. Fetch OHLCV data (cached)
Each call writes parquet files under `data/cache/raw`.

In [ ]:
ohlcv_data = {}
for symbol in tickers:
    ohlcv_data[symbol] = fetch_ohlcv(symbol)
list(ohlcv_data[tickers[0]].tail().itertuples())[:3]

## 2. Technical analysis + chart
Compute indicators/patterns and visualise one ticker.

In [ ]:
ta_data = {}
signals = {}
for symbol in tickers:
    enriched = compute_indicators(ohlcv_data[symbol])
    ta_data[symbol] = enriched
    symbol_signals = []
    for detector in (detect_breakout, detect_double_top_bottom):
        sig = detector(symbol, enriched)
        if sig:
            symbol_signals.append(sig)
    signals[symbol] = symbol_signals

sample = tickers[0]
latest = ta_data[sample].tail()
display(latest[["close", "sma_20", "ema_20", "rsi_14"]])

fig = go.Figure(data=[
    go.Candlestick(
        x=ta_data[sample].index,
        open=ta_data[sample]["open"],
        high=ta_data[sample]["high"],
        low=ta_data[sample]["low"],
        close=ta_data[sample]["close"],
        name=sample
    ),
])
fig.update_layout(height=400, title=f'{sample} price action')
fig.show()
signals[sample]

## 3. Fundamental summaries from Screener fallback
Scores combine valuation, growth, leverage, and profitability.

In [ ]:
fundamentals = {symbol: evaluate_fundamentals(symbol) for symbol in tickers}
fundamental_table = pd.DataFrame([
    {
        'ticker': f.ticker,
        'score': f.score,
        'roe': f.metrics.get('roe'),
        'pe': f.metrics.get('pe_ratio'),
        'debt_to_equity': f.metrics.get('debt_to_equity'),
    }
    for f in fundamentals.values()
])
fundamental_table.set_index('ticker')

## 4. Strategy scoring
Breakout/Swing/Intraday templates blend TA + FA.

In [ ]:
strategy_rows = []
strategy_boards = {}
for symbol in tickers:
    ranked = rank_strategies(symbol, ta_data[symbol], fundamentals[symbol], signals=signals[symbol])
    strategy_boards[symbol] = ranked
    for res in ranked:
        strategy_rows.append({
            'ticker': symbol,
            'strategy': res.strategy,
            'score': res.score,
            'entry': res.entry,
            'stop': res.stop,
            'target': res.target,
        })
pd.DataFrame(strategy_rows).sort_values(['score'], ascending=False).head(9)

## 5. Chat agent explanation
RAG context is pulled from the lightweight index.

In [ ]:
agent = ChatAgent()
best_pick = strategy_boards[sample][0]
print(agent.explain(sample, best_pick, fundamentals[sample]))

## 6. Next steps
Run the full web app locally:

In [ ]:
print('How to run locally:')
print('1. bash setup.sh')
print('2. source .venv/bin/activate')
print('3. uvicorn src.main:app --reload')
print('4. cd frontend && npm install && npm run dev')